# Aula 10_11 - Exercício com RAGAS


- Implementar o RAGAS com o LLaMA-3 70B para avaliar a qualidade das 50 anotações do IIRC usadas no exercício passado.
- O RAGAS considera context, question, answer, keys que estão disponíveis no conjunto de teste do IIRC.

Opcional:
- Avaliar as respostas do exercício da aula 9_10
- Usar multi agents

In [15]:
#@title Instalação de Dependências Necessárias para o Projeto

# Instala a biblioteca 'datasets', que contém ferramentas para manipular e carregar conjuntos de dados
!pip install datasets

# Instala a biblioteca 'ragas', utilizada para avaliação de respostas geradas por LLMs (Language Learning Models)
!pip install ragas

# Instala a biblioteca 'groq', que provavelmente oferece suporte à interface de modelos da Groq
!pip install groq

# Instala a biblioteca 'sentence-transformers', usada para calcular embeddings de texto e trabalhar com transformadores
!pip install sentence-transformers


In [16]:
# @title Importando as bibliotecas necessárias
from datasets import Dataset
from ragas.metrics import FaithfulnesswithHHEM
from ragas import evaluate
import json
from groq import Groq
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import torch
import os
import json
from abc import ABC, abstractmethod
import re
import plotly.graph_objects as go

In [17]:
#@title Carregando Dataset

# Verifica se a pasta 'data' existe, se não existir, cria.
if not os.path.isdir("data"):
    os.mkdir("data")

# Verifica se o arquivo context_articles.json existe, se não, faz o download e extrai corretamente.
if not os.path.isfile(os.path.join("data", "context_articles.json")):
    os.system('curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz')
    os.system('mv context_articles.tar.gz data/')
    os.system('tar -xf data/context_articles.tar.gz -C data')

    # A correção aqui é que o arquivo extraído será 'context_articles.json' na própria pasta 'data'
    if not os.path.isfile(os.path.join("data", "context_articles.json")):
        print("Erro ao mover ou descompactar context_articles.json")

# Verifica se o arquivo iirc_test.json existe, se não, faz o download.
if not os.path.isfile(os.path.join("data", "iirc_test.json")):
    os.system('curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json')
    os.system('mv iirc_test.json data/')

# Carrega os dados do arquivo context_articles.json
try:
    with open(os.path.join("data", "context_articles.json"), "r") as file:
        articles = json.load(file)
except FileNotFoundError:
    print("Erro: O arquivo 'context_articles.json' não foi encontrado.")

# Carrega os dados do arquivo iirc_test.json
try:
    with open(os.path.join("data", "iirc_test.json"), "r") as file:
        test_data = json.load(file)
except FileNotFoundError:
    print("Erro: O arquivo 'iirc_test.json' não foi encontrado.")


In [18]:
#@title Definindo questões

# Define o número de questões a serem processadas
n_question = 50

# Verifica se 'n_question' é um inteiro positivo
if not isinstance(n_question, int) or n_question <= 0:
    raise ValueError("O valor de 'n_question' deve ser um inteiro positivo.")

In [19]:
#@title Chave API Groq

class GroqModel():
    def __init__(self, api_key, model):
        # Inicializa a classe GroqModel com a chave da API e o modelo
        self.client = Groq(api_key=api_key)  # Conecta ao cliente da Groq usando a chave da API fornecida
        self.model = model  # Define o modelo a ser utilizado

    def get_answer(self, user_prompt):
        # Gera uma resposta com base no prompt do usuário
        response = self.client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": user_prompt,  # Mensagem enviada pelo usuário
                }
            ],
            model=self.model  # Utiliza o modelo definido na instância
        )
        # Retorna a resposta do modelo, removendo espaços em branco extras
        return response.choices[0].message.content.strip()

# Insira sua chave da API aqui e inicializa o modelo llama3 com a chave fornecida
llama3 = GroqModel(api_key='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', model="llama3-70b-8192")


In [20]:
#@title Classe Base Metric
# Classe pai que as métricas irão herdar
class Metric(ABC):

    def __init__(self, model):
        # Inicializa o modelo a ser utilizado pelas subclasses
        self.model = model

    # Método abstrato que obriga a implementação do método evaluate nas classes filhas
    @abstractmethod
    def evaluate(self, question, context, answer):
        raise NotImplementedError


In [21]:
#@title Classe Faithfulness para Avaliar a Fidelidade
class Faithfulness(Metric):
    def __init__(self, model):
        super().__init__(model)
        # Pré-compilar as expressões regulares
        self.statement_pattern = re.compile(r"Statement \d+:\s*(.+)")
        self.verdict_pattern = re.compile(r"Verdict \d+:\s*(Yes|No)")

    def __process_statements(self, statements):
        """
        Função para extrair afirmações geradas pela LLM
        """
        # Usa a expressão regular pré-compilada
        matches = self.statement_pattern.findall(statements)

        # Se não encontrar correspondências, retorna uma lista vazia diretamente
        return matches or []

    def __construct_verdict_prompt(self, statement_list, context):
        """
        Constrói o prompt para solicitar vereditos sobre as afirmações geradas
        """
        # Usar join para evitar concatenação de strings repetitivas
        statements_str = "\n".join([f"Statement {i+1}: {statement}" for i, statement in enumerate(statement_list)])
        verdict_format = "\n".join([f"Verdict {i+1}: Yes/No" for i in range(len(statement_list))])

        prompt = (
            f"Consider the given context and following\n"
            f"statements, then determine whether they\n"
            f"are supported by the information present\n"
            f"in the context. Provide a brief explanation for each statement before arriving\n"
            f"at the verdict (Yes/No). Provide a final\n"
            f"verdict for each statement in order at the\n"
            f"end in the given format. Do not deviate\n"
            f"from the specified format.\n\n"
            f"Context: {context}\n\n"
            f"{statements_str}\n\nFinal verdict format:\n{verdict_format}"
        )

        return prompt

    def __process_verdicts(self, verdicts):
        """
        Função para processar os vereditos gerados pela LLM
        """
        # Usa a expressão regular pré-compilada
        verdict_matches = self.verdict_pattern.findall(verdicts)

        # Usa list comprehension para converter vereditos em binários e calcular a média diretamente
        return sum(1 if v == 'Yes' else 0 for v in verdict_matches) / len(verdict_matches) if verdict_matches else 0

    def evaluate(self, question, answer, context):
        """
        Avalia a fidelidade da resposta em relação ao contexto
        """
        # Usa a LLM para obter as afirmações e os veredictos em uma única interação
        get_statements_prompt = f"""
        Given a question and answer, create one or more statements from each sentence in the given answer.
        Return the response in the following format:
        Statement 1: statement_1
        ...
        Statement n: statement_n
        Do not deviate from the specified format.

        question: {question}
        answer: {answer}
        """
        statements = self.model.get_answer(get_statements_prompt)

        # Processa as afirmações
        processed_statements = self.__process_statements(statements)
        if not processed_statements:
            return 0  # Retorna 0 diretamente se não houver afirmações processadas

        # Constrói o segundo prompt para avaliar a fidelidade
        faithfulness_prompt = self.__construct_verdict_prompt(processed_statements, context)

        # Obtém os vereditos da LLM
        verdicts = self.model.get_answer(faithfulness_prompt)

        # Processa os vereditos e calcula a fidelidade
        average_score = self.__process_verdicts(verdicts)

        return average_score


In [22]:
#@title Classe AnswerRelevance para Avaliar Relevância da Resposta

class AnswerRelevance(Metric):
    def __init__(self, model):
        super().__init__(model)
        # Define o dispositivo para cálculo dos embeddings
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

    def evaluate(self, question, answer, n=5):
        """
        Avalia a relevância da resposta gerando N perguntas a partir da resposta e comparando com a pergunta original
        """
        # Gera N questões a partir da resposta
        generated_questions = self.__generate_questions_from_answer(answer, n)

        # Calcula os embeddings
        original_question_embedding = self.embedding_model.encode(question, convert_to_tensor=True)
        generated_question_embeddings = self.embedding_model.encode(generated_questions, convert_to_tensor=True)

        # Calcula a similaridade por cosseno
        similarities = self.__calculate_similarities__(original_question_embedding, generated_question_embeddings)

        # Retorna a média das similaridades
        average_relevance_score = torch.mean(torch.tensor(similarities)).item()

        return average_relevance_score

    def __generate_questions_from_answer(self, answer, n):
        """
        Gera N perguntas a partir de uma resposta
        """
        generated_questions = []
        for _ in range(n):
            # Prompt para gerar perguntas
            generate_question_prompt = f"Generate a question for the given answer.\nanswer: {answer}"
            question = self.model.get_answer(generate_question_prompt)
            generated_questions.append(question.strip())

        return generated_questions

    def __calculate_similarities__(self, original_embedding, generated_embeddings):
        """
        Calcula a similaridade por cosseno entre a pergunta original e as perguntas geradas
        """
        original_embedding = original_embedding.unsqueeze(0)
        similarities = F.cosine_similarity(generated_embeddings, original_embedding, dim=1).tolist()
        return similarities


In [23]:
#@title Classe ContextRelevance para Avaliar Relevância do Contexto
class ContextRelevance(Metric):
    def __init__(self, model):
        super().__init__(model)

    def evaluate(self, question, context):
        """
        Avalia a relevância do contexto em relação à pergunta
        """
        # Extrai sentenças relevantes do contexto
        extracted_sentences = self.__extract_relevant_sentences(question, context)

        if "Insufficient Information" in extracted_sentences:
            relevance_score = 0.0
        else:
            relevance_score = len(extracted_sentences) / len(context)

        return relevance_score

    def __extract_relevant_sentences(self, question, context):
        """
        Extrai as sentenças relevantes do contexto que podem ajudar a responder à pergunta
        """
        # Junta o contexto em uma única string
        context_text = " ".join(context)

        # Prompt para extração de sentenças
        extract_prompt = f"""
        Please extract relevant sentences from the provided context that can potentially help answer the following question.
        If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information".
        While extracting candidate sentences, you’re not allowed to make any changes to sentences from the given context.

        question: {question}
        context: {context_text}
        """

        extracted_sentences = self.model.get_answer(extract_prompt)

        if "Insufficient Information" in extracted_sentences:
            return ["Insufficient Information"]
        else:
            # Divide as sentenças extraídas de volta em uma lista
            return self.__process_extracted_sentences(extracted_sentences)

    def __process_extracted_sentences(self, extracted_sentences):
        """
        Processa as sentenças, colocando-as em um array
        """
        list_of_sentences = [sentence.strip() for sentence in extracted_sentences.split('. ') if sentence]
        return list_of_sentences


In [24]:
#@title Inicialização das Métricas e Avaliação das Perguntas no Conjunto de Testes

# Inicializa as métricas com o modelo correto
faithfulness_metric = Faithfulness(llama3)  # Corrigido o nome da classe
answer_relevance_metric = AnswerRelevance(llama3)
context_relevance_metric = ContextRelevance(llama3)

# Dicionário para armazenar os resultados do conjunto de testes
test_set_results = {
    'questions': [],
    'contexts': [],
    'answers': [],
    'faithfulness': [],
    'answer_relevance': [],
    'context_relevance': []
}

# Certifique-se de que você tem uma lista de itens (como perguntas) para iterar
for idx, item in enumerate(test_data[:n_question]):  # Itera sobre os dados limitando a n_question
    data = item['questions'][0]

    # Extrai a resposta verdadeira de acordo com o tipo de resposta
    if data['answer']['type'] == 'span':
        true_answer = ", ".join([a['text'] for a in data['answer']["answer_spans"]])
    elif data['answer']['type'] == 'value':
        true_answer = "{0} {1}".format(data['answer']['answer_value'], data['answer']['answer_unit'])
    elif data['answer']['type'] == "binary":
        true_answer = data['answer']['answer_value']
    else:
        print("Answer type not supported")
        true_answer = ""

    # Extrai a pergunta e o contexto
    question = data['question']
    context = [elem['text'] for elem in data['context']]

    # Avalia cada métrica
    faithfulness = faithfulness_metric.evaluate(question, true_answer, context)
    answer_relevance = answer_relevance_metric.evaluate(question, true_answer, n=3)
    context_relevance = context_relevance_metric.evaluate(question, context)

    # Armazena os resultados
    test_set_results['questions'].append(question)
    test_set_results['contexts'].append(context)
    test_set_results['answers'].append(true_answer)
    test_set_results['faithfulness'].append(faithfulness)
    test_set_results['answer_relevance'].append(answer_relevance)
    test_set_results['context_relevance'].append(context_relevance)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported
Answer type not supported


In [25]:
#@title Resultados do Conjunto de Testes

# Inicializa uma variável para armazenar os resultados do conjunto de testes
test_set_results

{'questions': ['What is Zeus know for in Greek mythology?',
  'How long had the First World War been over when Messe was named aide-de-camp?',
  'How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty\'s Secret Service"?',
  'What is the capacity of the stadium where Brunt returned to action after a torn ACL?',
  'In which country was Wilhelm Müller born?',
  'In which Italian region did Pesce studied medicine?',
  'What albums were ranked higher than "It Takes a Nation of Millions to Hold Us Back" in Rolling Stone\'s the 500 Greatest Albums of All Time?',
  'When was the sports organization that the Turks and Caicos Islands became affiliate members in 2002 founded?',
  'When was the port established at the Port Phillip District?',
  'At which tournament were more goals scored, 1984 Canada Cup or 1994 Ice Hockey World Championship?',
  'How much money did IBM earn the year Delicious was founded?',
  'How many years had Mike Elizondo been a

In [26]:
#@title Exibição dos Resultados do Conjunto de Testes

# Itera sobre os resultados armazenados e imprime os dados correspondentes para cada pergunta
for i in range(len(test_set_results['questions'])):

  # Imprime a pergunta
  print(f'Pergunta: {test_set_results["questions"][i]}')

  # Imprime o contexto associado
  print(f'Contexto: {test_set_results["contexts"][i]}')

  # Imprime a(s) resposta(s)
  print(f'Respostas: {test_set_results["answers"][i]}')

  # Imprime a avaliação de fidelidade
  print(f'Fidelidade: {test_set_results["faithfulness"][i]}')

  # Imprime a relevância da resposta
  print(f'Relevância da Resposta: {test_set_results["answer_relevance"][i]}')

  # Imprime a relevância do contexto
  print(f'Relevância do Contexto: {test_set_results["context_relevance"][i]}')

  # Linha em branco para separação entre resultados
  print()


Pergunta: What is Zeus know for in Greek mythology?
Contexto: ['he Palici the sons of Zeus', 'in Greek mythology', 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion']
Respostas: sky and thunder god
Fidelidade: 1.0
Relevância da Resposta: 0.40795910358428955
Relevância do Contexto: 0.3333333333333333

Pergunta: How long had the First World War been over when Messe was named aide-de-camp?
Contexto: ['he became aide-de-camp to King Victor Emmanuel III, holding this post from 1923 to 1927', 'a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918']
Respostas: 5 years
Fidelidade: 1.0
Relevância da Resposta: 0.1731971949338913
Relevância do Contexto: 0.5

Pergunta: How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
Contexto: ["Angela Scoular appeared as Ruby Bartlett in On Her Majesty's Secret Service", "laying Ruby in 1969's On Her 

In [30]:
#@title Cálculo das Médias das Métricas no Conjunto de Testes"

print(50*"-")
print("Médias das Métricas do Conjunto de Testes")
print(50*"-", "\n")

# Calcula e imprime a média de Fidelidade do conjunto de testes
print(f'Média de Fidelidade: {sum(test_set_results["faithfulness"]) / len(test_set_results["faithfulness"])}')

# Calcula e imprime a média de Relevância da Resposta do conjunto de testes
print(f'Média de Relevância da Resposta: {sum(test_set_results["answer_relevance"]) / len(test_set_results["answer_relevance"])}')

# Calcula e imprime a média de Relevância do Contexto do conjunto de testes
print(f'Média de Relevância do Contexto: {sum(test_set_results["context_relevance"]) / len(test_set_results["context_relevance"])}\n')

print(50*"-", "\n")

# Calcula as médias das métricas
average_faithfulness = sum(test_set_results["faithfulness"]) / len(test_set_results["faithfulness"])
average_answer_relevance = sum(test_set_results["answer_relevance"]) / len(test_set_results["answer_relevance"])
average_context_relevance = sum(test_set_results["context_relevance"]) / len(test_set_results["context_relevance"])

# Prepara os dados para o gráfico
metrics = ['Fidelidade', 'Relevância da Resposta', 'Relevância do Contexto']
average_scores = [average_faithfulness, average_answer_relevance, average_context_relevance]

# Cria o gráfico de barras com Plotly
fig = go.Figure(data=[
    go.Bar(name='Média', x=metrics, y=average_scores)
])

# Atualiza o layout do gráfico
fig.update_layout(
    title="Média das Métricas no Conjunto de Testes",
    xaxis_title="Métricas",
    yaxis_title="Média",
    yaxis=dict(range=[0, 1]),  # Define o intervalo do eixo y entre 0 e 1
    template="plotly"
)

# Mostra o gráfico
fig.show()

--------------------------------------------------
Médias das Métricas do Conjunto de Testes
-------------------------------------------------- 

Média de Fidelidade: 0.6
Média de Relevância da Resposta: 0.2178699977695942
Média de Relevância do Contexto: 0.36899999999999994

-------------------------------------------------- 

